In [88]:
import pandas as pd
import numpy as np

import re
import math

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve
from catboost import Pool, CatBoostClassifier

from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations
from statsmodels.stats.proportion import proportion_confint

Use the Lending Club dataset provided by Nathan George that has all Lending Club loan features. This dataset contains more than 2 million rows so to reduce the processing times, I only selected the loans issued in 2018 (≈0.5 million rows)

## Data Preprocessing

### 1. all the available features

In [3]:
data = pd.read_csv(
    '/Users/jinxiaowei/Desktop/Data Science prep/github/min_risk/minimize_risk/accepted_2007_to_2018Q4.csv',
    parse_dates=['issue_d'], infer_datetime_format=True)
data = data[(data.issue_d >= '2018-01-01 00:00:00') & (data.issue_d < '2019-01-01 00:00:00')]
data = data.reset_index(drop=True)
data.head()

/Users/jinxiaowei/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,130954621,NaN,5000.0,5000.0,5000.0,36 months,20.39,186.82,D,D4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,130964697,NaN,15000.0,15000.0,15000.0,36 months,9.92,483.45,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,130955326,NaN,11200.0,11200.0,11200.0,60 months,30.79,367.82,G,G1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,130504052,NaN,25000.0,25000.0,25000.0,60 months,21.85,688.35,D,D5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,130956066,NaN,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,A4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


This dataset contains more than 150 features but some of them are only relevant after the loan is issued and therefore, not available at the moment of investing. To get the list of features that are visible to investors, I use the Lending Club Data Dictionary provided by Wendy Kan (the sheet called "Browse Notes")

In [4]:
browse_notes = pd.read_excel('./lending-club-loan-data/LCDataDictionary.xlsx',
                             sheet_name=1)
browse_notes.head()

,BrowseNotesFile,Description
0,acceptD,The date which the borrower accepted the offer
1,accNowDelinq,The number of accounts on which the borrower i...
2,accOpenPast24Mths,Number of trades opened in past 24 months.
3,addrState,The state provided by the borrower in the loan...
4,all_util,Balance to credit limit on all trades


The missing lines due to Excel formatting are removed and the feature names are saved in browse_feat. Some of them, however, have a different spelling format from the one used in the loan dataset. This format uses capital letters instead of underscores so I identify them using using regular expressions and then correct them. For some features there are whitespaces in front of their names which I remove as well.

In [5]:
browse_feat = browse_notes['BrowseNotesFile'].dropna().values
browse_feat = [re.sub('(?<![0-9_])(?=[A-Z0-9])', '_', x).lower().strip() for x in browse_feat]

In [12]:
browse_feat

array(['acc_now_delinq', 'acc_open_past_24mths', 'accept_d', 'addr_state',
       'all_util', 'annual_inc', 'annual_inc_joint', 'application_type',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'collections_12_mths_ex_med',
       'credit_pull_d', 'delinq_2yrs', 'delinq_amnt', 'desc',
       'disbursement_method', 'dti', 'dti_joint', 'earliest_cr_line',
       'effective_int_rate', 'emp_length', 'emp_title', 'exp_d',
       'exp_default_rate', 'fico_range_high', 'fico_range_low',
       'funded_amnt', 'grade', 'home_ownership', 'id', 'il_util',
       'ils_exp_d', 'initial_list_status', 'inq_fi', 'inq_last_12m',
       'inq_last_6mths', 'installment', 'int_rate', 'list_d', 'loan_amnt',
       'max_bal_bc', 'member_id', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'msa',
       'mths_since_last_delinq', 'mths_since_last_major_derog',
       'mths_since_last_record', 'mths_since_rcnt_il',
       'mths_since_

Still not all the features from "Browse Notes" browse_feat could be matched with the original features data_feat so I print out the unmatched features from both lists to see if some of them could be matched manually.

In [8]:
data_feat = data.columns.values
np.setdiff1d(browse_feat, data_feat)

array(['accept_d', 'credit_pull_d', 'effective_int_rate', 'exp_d',
       'exp_default_rate', 'ils_exp_d', 'is_inc_v', 'list_d', 'msa',
       'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
       'mths_since_recent_loan_delinq', 'review_status',
       'review_status_d', 'service_fee_rate', 'verified_status_joint'],
      dtype='<U35')

In [9]:
np.setdiff1d(data_feat, browse_feat)

array(['collection_recovery_fee', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'deferral_term', 'funded_amnt_inv',
       'hardship_amount', 'hardship_dpd', 'hardship_end_date',
       'hardship_flag', 'hardship_last_payment_amount', 'hardship_length',
       'hardship_loan_status', 'hardship_payoff_balance_amount',
       'hardship_reason', 'hardship_start_date', 'hardship_status',
       'hardship_type', 'issue_d', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt',
       'last_pymnt_d', 'loan_status', 'mo_sin_old_il_acct',
       'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
       'next_pymnt_d', 'orig_projected_additional_accrued_interest',
       'out_prncp', 'out_prncp_inv', 'payment_plan_start_date',
       'policy_code', 'pymnt_plan', 'recoveries', 'settlement_amount',
       'settlement_date', 'settlement_percentage', 'settlement_status',
       'settlement_term', 'total_pymnt', 'total_pymnt_inv',
       'total

Indeed some of the features are spelled differently but mean the same thing, for example verified_status_joint and verification_status_joint. So I remove wrong and add correct ones to the list browse_feat.

In [10]:
wrong = ['is_inc_v', 'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
         'mths_since_recent_loan_delinq', 'verified_status_joint']
correct = ['verification_status', 'mths_since_recent_inq', 'mo_sin_old_il_acct',
           'mths_since_recent_bc_dlq', 'verification_status_joint']

browse_feat = np.setdiff1d(browse_feat, wrong)
browse_feat = np.append(browse_feat, correct)

Finally all matching features are saved in the list avail_feat a new DataFrame X that only contains these features, is created. It is a good practice to set every newly created DataFrame as a copy in order to avoid hidden chained assignments and SettingWithCopyWarning further down the code.

In [11]:
avail_feat = np.intersect1d(browse_feat, data_feat)
X = data[avail_feat].copy()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495242 entries, 0 to 495241
Columns: 109 entries, acc_now_delinq to zip_code
dtypes: float64(89), object(20)
memory usage: 411.8+ MB


### 2.Feature Types

In [13]:
X.select_dtypes('object').head()

,addr_state,application_type,desc,disbursement_method,earliest_cr_line,emp_length,emp_title,grade,home_ownership,id,initial_list_status,purpose,sec_app_earliest_cr_line,sub_grade,term,title,url,verification_status,verification_status_joint,zip_code
0,OK,Individual,NaN,Cash,Jan-2009,8 years,General Manager,D,RENT,130954621,w,other,NaN,D4,36 months,Other,https://lendingclub.com/browse/loanDetail.acti...,Verified,NaN,740xx
1,FL,Individual,NaN,Cash,Jul-1998,2 years,IT Director,B,OWN,130964697,w,debt_consolidation,NaN,B2,36 months,Debt consolidation,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,337xx
2,NH,Joint App,NaN,Cash,Jul-2007,< 1 year,Client services,G,RENT,130955326,w,medical,Feb-2005,G1,60 months,Medical expenses,https://lendingclub.com/browse/loanDetail.acti...,Not Verified,Not Verified,030xx
3,AL,Individual,NaN,Cash,Mar-1995,10+ years,Asphalt Supervisor,D,MORTGAGE,130504052,w,debt_consolidation,NaN,D5,60 months,Debt consolidation,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,361xx
4,WA,Individual,NaN,Cash,Jan-1998,9 years,Scale Technician,A,RENT,130956066,w,major_purchase,NaN,A4,36 months,Major purchase,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,988xx


The features `earlist_cr_line` and `sec_app_earliest_cr_line` are dates and their type should be changed to `datetime`

In [14]:
X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
X['sec_app_earliest_cr_line'] = pd.to_datetime(X['sec_app_earliest_cr_line'], infer_datetime_format=True)

In [18]:
X['earliest_cr_line'].head()

0   2009-01-01
1   1998-07-01
2   2007-07-01
3   1995-03-01
4   1998-01-01
Name: earliest_cr_line, dtype: datetime64[ns]

In [19]:
X['sec_app_earliest_cr_line'].head()

0          NaT
1          NaT
2   2005-02-01
3          NaT
4          NaT
Name: sec_app_earliest_cr_line, dtype: datetime64[ns]

The features `emp_length` and `id` are numeric and their type should be changed to `float`
For `emp_length`, replace the extreme cases:
* `< 1 year` with `0 years`
* `10+ year` with `11 years`

In [20]:
# mapping values
X['emp_length'] = X['emp_length'].replace({'< 1 year': '0 years', '10+ years': '11 years'})
# transform types
X['emp_length'] = X['emp_length'].str.extract('(\d+)').astype('float')
X['id'] = X['id'].astype('float')

In [21]:
X['emp_length'].head()

0     8.0
1     2.0
2     0.0
3    11.0
4     9.0
Name: emp_length, dtype: float64

In [22]:
X['id'].head()

0    130954621.0
1    130964697.0
2    130955326.0
3    130504052.0
4    130956066.0
Name: id, dtype: float64

### 3. Missing Values

In [23]:
X.isnull().sum()

acc_now_delinq                              0
acc_open_past_24mths                        0
addr_state                                  0
all_util                                  129
annual_inc                                  0
annual_inc_joint                       426257
application_type                            0
avg_cur_bal                                40
bc_open_to_buy                           6588
bc_util                                  6803
chargeoff_within_12_mths                    0
collections_12_mths_ex_med                  0
delinq_2yrs                                 0
delinq_amnt                                 0
desc                                   495242
disbursement_method                         0
dti                                      1132
dti_joint                              426257
earliest_cr_line                            0
emp_length                              41987
emp_title                               54659
fico_range_high                   

The ratio of missing value for each column:

In [26]:
na_ratio = X.isna().mean()
na_ratio

acc_now_delinq                         0.000000
acc_open_past_24mths                   0.000000
addr_state                             0.000000
all_util                               0.000260
annual_inc                             0.000000
annual_inc_joint                       0.860704
application_type                       0.000000
avg_cur_bal                            0.000081
bc_open_to_buy                         0.013303
bc_util                                0.013737
chargeoff_within_12_mths               0.000000
collections_12_mths_ex_med             0.000000
delinq_2yrs                            0.000000
delinq_amnt                            0.000000
desc                                   1.000000
disbursement_method                    0.000000
dti                                    0.002286
dti_joint                              0.860704
earliest_cr_line                       0.000000
emp_length                             0.084781
emp_title                              0

In [27]:
na_ratio = na_ratio[na_ratio != 0].sort_values()

In [28]:
na_ratio

pct_tl_nvr_dlq                         0.000004
avg_cur_bal                            0.000081
all_util                               0.000260
revol_util                             0.001195
dti                                    0.002286
mths_since_recent_bc                   0.012515
bc_open_to_buy                         0.013303
percent_bc_gt_75                       0.013319
bc_util                                0.013737
num_tl_120dpd_2m                       0.025046
mo_sin_old_il_acct                     0.037174
mths_since_rcnt_il                     0.037174
emp_length                             0.084781
emp_title                              0.110368
mths_since_recent_inq                  0.123788
il_util                                0.163201
mths_since_last_delinq                 0.558620
mths_since_recent_revol_delinq         0.711878
mths_since_last_major_derog            0.768128
mths_since_recent_bc_dlq               0.801895
sec_app_chargeoff_within_12_mths       0

Remove `desc` and `member_id` since no information is provided

In [29]:
X = X.drop(['desc', 'member_id'], axis = 1, errors = 'ignore')

In [30]:
X.head()

,acc_now_delinq,acc_open_past_24mths,addr_state,all_util,annual_inc,annual_inc_joint,application_type,avg_cur_bal,bc_open_to_buy,bc_util,...,total_bal_ex_mort,total_bal_il,total_bc_limit,total_cu_tl,total_il_high_credit_limit,total_rev_hi_lim,url,verification_status,verification_status_joint,zip_code
0,0.0,3.0,OK,58.0,50000.0,NaN,Individual,3869.0,384.0,23.2,...,19344.0,14118.0,500.0,0.0,27820.0,500.0,https://lendingclub.com/browse/loanDetail.acti...,Verified,NaN,740xx
1,0.0,15.0,FL,51.0,196000.0,NaN,Individual,31468.0,7368.0,74.1,...,137713.0,113470.0,28500.0,1.0,147178.0,52400.0,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,337xx
2,0.0,5.0,NH,84.0,44000.0,81000.0,Joint App,8397.0,632.0,66.7,...,67173.0,65647.0,1900.0,1.0,74167.0,6200.0,https://lendingclub.com/browse/loanDetail.acti...,Not Verified,Not Verified,030xx
3,0.0,2.0,AL,90.0,65000.0,NaN,Individual,10685.0,63.0,98.1,...,17039.0,8382.0,3300.0,3.0,10220.0,8800.0,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,361xx
4,0.0,3.0,WA,1.0,52000.0,NaN,Individual,25099.0,30359.0,0.5,...,141.0,0.0,30500.0,2.0,0.0,31000.0,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,988xx


For categorical features `emp_title`, `verification_status_joint` the missing values should be filled with an empty string

In [33]:
fillna_empty = ['emp_title', 'verification_status_joint']
X[fillna_empty] = X[fillna_empty].fillna('')

For some of the numeric features the missing values should be filled using the maximum value of the respective columns so these features are placed in the list `fill_max`. For example, the feature `mths_since_last_record` indicates the number of months since the last record (like bankruptcy, foreclosure, tax liens, etc.) so if missing, one should assume that no records were made and the number of months since the "last" record should be a maximum.

In [34]:
fillna_max = ['bc_open_to_buy', 'mo_sin_old_il_acct', 'mths_since_last_delinq',
            'mths_since_last_major_derog', 'mths_since_last_record',
            'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
            'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
            'pct_tl_nvr_dlq','sec_app_mths_since_last_major_derog']

In [35]:
X[fillna_max] = X[fillna_max].fillna(X[fillna_max].max())

For the rest of the numeric features the missing values should be filled using the minimum value of the respective columns so these features are placed in the list `fill_min`. For example, the feature `emp_length` indicates employment length. So if missing, one should assume that the borrower never worked and the number of working years should be a minimum.

In [37]:
fillna_min = np.setdiff1d(X.columns.values, np.append(fillna_empty,fillna_max))

In [39]:
X[fillna_min] = X[fillna_min].fillna(X[fillna_min].min())

In [45]:
# check if all the n/a are filled or not
print(X.isnull().sum().nunique())
print(X.isnull().sum()[0])

1
0


### 4. Correlated Features

* check unique values for numerical columns first
* then check categorical features

#### 4.1 Numerical Values

In [46]:
num_features = X.select_dtypes('number').columns.values

In [47]:
X[num_features].nunique().sort_values()

num_tl_120dpd_2m                           1
acc_now_delinq                             2
num_tl_30dpd                               2
inq_last_6mths                             6
sec_app_inq_last_6mths                     7
open_il_12m                                8
pub_rec_bankruptcies                       8
chargeoff_within_12_mths                   9
collections_12_mths_ex_med                 9
emp_length                                11
open_acc_6m                               15
pub_rec                                   16
tax_liens                                 16
sec_app_collections_12_mths_ex_med        17
open_il_24m                               21
sec_app_chargeoff_within_12_mths          21
sec_app_mort_acc                          22
open_rv_12m                               24
num_tl_90g_dpd_24m                        25
delinq_2yrs                               26
mths_since_recent_inq                     26
num_tl_op_past_12m                        26
mort_acc  

Remove `num_tl_120dpd_2m` since it is a constant.

In [48]:
X = X.drop(X['num_tl_120dpd_2m'], axis = 1, errors = 'ignore')

Remove `id` since that is identical to every single person. That will potentially cause overfitting.

In [49]:
X = X.drop(X['id'], axis = 1, errors = 'ignore')

For all pairs of numerical values, calculate their Pearson's R correlation coefficient, and store it in `corr_num_feat`

In [50]:
num_feat = X.select_dtypes('number').columns.values
# print the list of all the possible combinations
comb_num_feat = np.array(list(combinations(num_feat, 2)))
corr_num_feat = np.array([])
for comb in comb_num_feat:
    corr = pearsonr(X[comb[0]], X[comb[1]])[0]
    corr_num_feat = np.append(corr_num_feat, corr)

/Users/jinxiaowei/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [51]:
comb_num_feat

array([['acc_now_delinq', 'acc_open_past_24mths'],
       ['acc_now_delinq', 'all_util'],
       ['acc_now_delinq', 'annual_inc'],
       ...,
       ['total_cu_tl', 'total_il_high_credit_limit'],
       ['total_cu_tl', 'total_rev_hi_lim'],
       ['total_il_high_credit_limit', 'total_rev_hi_lim']], dtype='<U35')

In [52]:
corr_num_feat

array([0.00059518, 0.00157713, 0.0022233 , ..., 0.17340478, 0.08317638,
       0.18542588])

Find out all the high correlation pairs, i.e correlation coefficient >= 0.9

In [53]:
print(np.abs(corr_num_feat) >= 0.9)

[False False False ... False False False]


/Users/jinxiaowei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater_equal
  """Entry point for launching an IPython kernel.


In [54]:
high_corr_num = comb_num_feat[np.abs(corr_num_feat) >= 0.9]

/Users/jinxiaowei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater_equal
  """Entry point for launching an IPython kernel.


In [55]:
high_corr_num

array([['acc_now_delinq', 'num_tl_30dpd'],
       ['fico_range_high', 'fico_range_low'],
       ['funded_amnt', 'installment'],
       ['funded_amnt', 'loan_amnt'],
       ['installment', 'loan_amnt'],
       ['mo_sin_old_il_acct', 'mths_since_rcnt_il'],
       ['num_actv_rev_tl', 'num_rev_tl_bal_gt_0'],
       ['num_sats', 'open_acc'],
       ['sec_app_fico_range_high', 'sec_app_fico_range_low'],
       ['sec_app_num_rev_accts', 'sec_app_open_acc'],
       ['tot_cur_bal', 'tot_hi_cred_lim'],
       ['total_bal_ex_mort', 'total_bal_il'],
       ['total_bal_il', 'total_il_high_credit_limit']], dtype='<U35')

Drop one of these features in the high corrleation pairs.

In [57]:
# avoid delete the same ones
np.unique(high_corr_num[:,0])

array(['acc_now_delinq', 'fico_range_high', 'funded_amnt', 'installment',
       'mo_sin_old_il_acct', 'num_actv_rev_tl', 'num_sats',
       'sec_app_fico_range_high', 'sec_app_num_rev_accts', 'tot_cur_bal',
       'total_bal_ex_mort', 'total_bal_il'], dtype='<U35')

In [58]:
X = X.drop(np.unique(high_corr_num[:,0]), axis=1, errors ='ignore')

#### 4.2 Caregorical Values

In [61]:
cate_feat = X.select_dtypes('object').columns.values

In [62]:
X[cate_feat].nunique().sort_values()

application_type                  2
disbursement_method               2
initial_list_status               2
term                              2
verification_status               3
home_ownership                    4
verification_status_joint         4
grade                             7
title                            12
purpose                          13
sub_grade                        35
addr_state                       50
zip_code                        897
emp_title                    129450
url                          495242
dtype: int64

The same reason as `id` for numerical values, we delete `url` to avoid overfitting.

In [63]:
X = X.drop(X['url'], axis=1, errors='ignore')

`emp_title` has a large number of unique values, so remove it.

In [64]:
X = X.drop(X['emp_title'], axis=1, errors='ignore')

For all pairs of the categorical features `comb_cat_feat`, calculate the Cramer's V correlateion coefficient that is expressed through the chi-square statistic $\chi^2$ of the contingency table: 
$$
V = \sqrt\frac{\chi^2}{n(min(K1,K2)-1)}
$$

where $n$ is the sum of all elements in the contingency table, $K1$ and $K2$ are the dimensions of the contingency table. note that Pearson's R correlation coefficient isn't applicable to categorical features.

In [66]:
cat_feat = X.select_dtypes('object').columns.values
comb_cat_feat = np.array(list(combinations(cat_feat, 2)))
corr_cat_feat = np.array([])